In [ ]:
import os
import sys

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()
sys.path.append(os.path.abspath("Efficient-Computing/Detection/Gold-YOLO"))

OUTPUT_DIR = "../data/others/model_output"

folder_paths = [OUTPUT_DIR]

# Create the output directories if they doesn't exist
for folder_path in folder_paths:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from plot import get_bounding_boxes, create_bboxes_image
import cv2
import numpy as np
from layers import AMF_GD_YOLOv8
import matplotlib.pyplot as plt
from time import time
import random
import torch
import torch.nn as nn
from utils import Folders
from data_processing import ImageData
from tqdm.notebook import tqdm
from typing import Sequence, Dict, Any, Callable, List, Tuple
import os
from torch.utils.data import Dataset, DataLoader
from utils import get_file_base_name
from skimage import io
from copy import deepcopy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = (
    "garbage_collection_threshold:0.6,max_split_size_mb:512"
)
print(os.environ.get("PYTORCH_CUDA_ALLOC_CONF"))

### Run the YOLOv8 model

In [ ]:
from ultralytics.models.yolo import YOLO
from ultralytics.models.yolo.detect.predict import DetectionPredictor

# Load a pretrained YOLO model
model = YOLO("yolov8n.pt")
# results = model.train(data="coco8.yaml", epochs=3, verbose=False)

# Perform object detection on an image using the model
results = model.predict("https://ultralytics.com/images/bus.jpg", save=True)

### Use the AMF GD YOLOv8 model

In [ ]:
from typing import Dict
from ultralytics.engine.trainer import BaseTrainer
from ultralytics.models.yolo.detect.train import DetectionTrainer
from ultralytics.utils import DEFAULT_CFG
from ultralytics.utils.loss import v8DetectionLoss


# class CustomTrainer(BaseTrainer):
#     def __init__(
#         self, cfg=DEFAULT_CFG, overrides=None, _callbacks=None
#     ):
#         super().__init__(cfg, overrides, _callbacks)
#         self.model = self.args.model

#     def get_model(self, cfg=None, weights=None, verbose=True):
#         # model = DetectionModel(cfg, nc=self.data["nc"], verbose=verbose and RANK == -1)
#         # if weights:
#         #     model.load(weights)
#         # return model
#         return self.model


class_names = {
    0: "Tree",
    1: "Tree_unsure",
    2: "Tree_disappeared",
    3: "Tree_replaced",
    4: "Tree_new",
}

class_indices = {value: key for key, value in class_names.items()}

# model = AMF_GD_YOLOv8(3, 1, device=device, scale="s", class_names=class_names)

# overrides = {"model": model, "data": "coco8.yaml"}


class TestModel(AMF_GD_YOLOv8):
    def __init__(
        self,
        c_input_left: int,
        c_input_right: int,
        class_names: Dict[int, str],
        device: torch.device,
        scale: str = "n",
        r: int = 16,
        gd_config_file: str | None = None,
    ) -> None:
        super().__init__(
            c_input_left, c_input_right, class_names, device, scale, r, gd_config_file
        )

        class Args:
            def __init__(self) -> None:
                self.box = 1.0
                self.cls = 1.0
                self.dfl = 1.0

        self.args = Args()
        self.model = nn.ModuleList([self.amfnet, self.gd, self.detect])


# class v8DetectionLossTest(v8DetectionLoss):
#     def __init__(self, model):
#         """Initializes v8DetectionLoss with the model, defining model-related properties and BCE loss function."""
#         device = next(model.parameters()).device  # get model device
#         h = model.args  # hyperparameters

#         m = model.model[-1]  # Detect() module
#         self.bce = nn.BCEWithLogitsLoss(reduction="none")
#         self.hyp = h
#         self.stride = m.stride  # model strides
#         self.nc = m.nc  # number of classes
#         self.no = m.nc + m.reg_max * 4
#         self.reg_max = m.reg_max
#         self.device = device

#         self.use_dfl = m.reg_max > 1

#         self.assigner = TaskAlignedAssigner(topk=10, num_classes=self.nc, alpha=0.5, beta=6.0)
#         self.bbox_loss = BboxLoss(m.reg_max - 1, use_dfl=self.use_dfl).to(device)
#         self.proj = torch.arange(m.reg_max, dtype=torch.float, device=device)


model_test = TestModel(3, 1, device=device, scale="s", class_names=class_names)

criterion = v8DetectionLoss(model_test)

# results = model.train(data="coco8.yaml", trainer=CustomTrainer, epochs=3, overrides=overrides)

# rgb_image_path = "../data/images/cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_1_1.tif"
# chm_image_path = (
#     "../data/CHM_cropped_0p24/122000_484000/122000_484000_1_1_filtered_chm.tif"
# )
# annotations_path = "../data/annotations/cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_1_1.json"

# gt_bboxes, gt_classes = get_bounding_boxes(annotations_path)

# output_name = "Model_output_test.jpg"
# output_path = os.path.join(OUTPUT_DIR, output_name)

# model.train()
# output = model(rgb_image_path, chm_image_path, image_save_path=output_path)

In [ ]:
# from ultralytics.models.utils.loss import DETRLoss
# from layers import xywh2xyxy

# nc = 4
# loss_func = DETRLoss(nc=nc)

# # output[0][0][0, :, 0] = torch.Tensor([107.3209, 151.2296, 161.4145, 207.6499, 0.9837, 0.5472, 0.3150, 0.5183])

# pred_bboxes = output[0][0].unsqueeze(0).permute((0, 1, 3, 2))[:, :, :, :4]
# pred_scores = output[0][0].unsqueeze(0).permute((0, 1, 3, 2))[:, :, :, 4:]
# batch = {
#     "cls": torch.Tensor([class_indices[cls] for cls in classes]).to(dtype=torch.int64),
#     "bboxes": xywh2xyxy(torch.Tensor(bboxes)),
#     "gt_groups": [len(classes)],
# }

# prefect_pred_bboxes = batch["bboxes"].unsqueeze(0).unsqueeze(0)
# prefect_pred_scores = (
#     torch.where(
#         batch["cls"].unsqueeze(-1).repeat(1, nc)
#         == torch.arange(nc).unsqueeze(0).repeat(batch["cls"].shape[0], 1),
#         1.0,
#         -1.0,
#     )
#     .unsqueeze(0)
#     .unsqueeze(0)
# )

# loss = loss_func.forward(pred_bboxes, pred_scores, batch)
# print(f"{loss = }")

model.train()
for i in range(100):
    output = model(rgb_image_path, chm_image_path, image_save_path=output_path)
    loss = model.compute_loss(output[0], gt_bboxes, gt_classes)
    print(loss)
    (loss["loss_class"] + loss["loss_bbox"]).backward()

In [ ]:
num_epochs = 100

dataloader = ...
model = AMF_GD_YOLOv8(3, 1, scale="n", class_names=class_names)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lambda i: 1 / np.sqrt(i + 2), last_epoch=-1, verbose=True
)

model.train()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        rgb_image = ...
        chm_image = ...

        output = model(rgb_image, chm_image)
        loss = model.compute_loss(output[0], gt_bboxes, gt_classes)
        # total_loss = loss["loss_class"] + loss["loss_bbox"]
        total_loss = torch.sum(torch.stack(list(loss.values())), dim=0).float()

        optimizer.zero_grad()

        total_loss.backward()

        optimizer.step()

    scheduler.step()
    print(f"Loss class: {loss["loss_class"]:.4f}, Loss bbox: {loss["loss_bbox"]:.4f}")

### How to use data augmentation with Albumentations

In [ ]:
class Pad(A.PadIfNeeded):
    def __init__(
        self,
        added_height: int | None = 64,
        added_width: int | None = 64,
        pad_height_divisor: int | None = None,
        pad_width_divisor: int | None = None,
        position: A.PadIfNeeded.PositionType | str = A.PadIfNeeded.PositionType.CENTER,
        border_mode: int = cv2.BORDER_REFLECT_101,
        value: float | Sequence[float] | None = None,
        mask_value: float | Sequence[float] | None = None,
        always_apply: bool = False,
        p: float = 1,
    ):
        super().__init__(
            0,
            0,
            pad_height_divisor,
            pad_width_divisor,
            position,
            border_mode,
            value,
            mask_value,
            always_apply,
            p,
        )
        self.added_height = added_height
        self.added_width = added_width

    def update_params(self, params: Dict[str, Any], **kwargs: Any) -> Dict[str, Any]:
        params = super().update_params(params, **kwargs)
        rows = params["rows"]
        cols = params["cols"]
        self.min_height = rows + self.added_height
        self.min_width = cols + self.added_width
        return super().update_params(params, **kwargs)

In [ ]:
crop_size = 640
distort_steps = 30
distort_limit = 0.2

bbox_params = A.BboxParams(
    format="pascal_voc", min_area=0, min_visibility=0.2, label_fields=["class_labels"]
)

transform_spatial = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.GridDistortion(
            num_steps=distort_steps,
            distort_limit=(-distort_limit, distort_limit),
            border_mode=cv2.BORDER_CONSTANT,
            normalized=True,
            p=0.5,
        ),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.Perspective(interpolation=cv2.INTER_LINEAR, p=0.25),
    ],
    bbox_params=bbox_params,
)

transform_pixel = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        # A.Normalize(p=1.0),
    ],
)

image_path = "../data/images/cropped/2023_122000_484000_RGB_hrl/0_0_640_640.tif"
annotations_path = (
    "../data/annotations/cropped/2023_122000_484000_RGB_hrl/0_0_640_640.json"
)
# image_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
# annotations_path = "../data/annotations/full/10"

t = time()

image = cv2.imread(image_path)
bboxes, labels = get_bounding_boxes(annotations_path)

print(f"Time elapsed: {round(time() - t, 3)}")

number_tests = 10

colors = {
    "Tree": (104, 201, 45),
    "Tree_unsure": (255, 215, 158),
    "Tree_disappeared": (158, 174, 255),
    "Tree_replaced": (255, 90, 82),
    "Tree_new": (251, 106, 225),
}

t = time()

for i in range(number_tests):
    transformed_spatial = transform_spatial(
        image=image, bboxes=[bbox.as_list() for bbox in bboxes], class_labels=labels
    )
    transformed_spatial_image = transformed_spatial["image"]
    transformed_bboxes = transformed_spatial["bboxes"]
    transformed_class_labels = transformed_spatial["class_labels"]
    transformed = transform_pixel(image=transformed_spatial_image)
    transformed_image = transformed["image"]

    output_name = f"Augmentation_test_{i}.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, transformed_image)

    bboxes_image = create_bboxes_image(
        transformed_image,
        transformed_bboxes,
        labels=transformed_class_labels,
        colors_dict=colors,
        scores=np.random.rand((len(bboxes))),
    )

    output_name = f"Augmentation_test_{i}_bboxes.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, bboxes_image)

print(f"Time elapsed: {round(time() - t, 3)}")

### PyTorch dataset

In [ ]:
full_image_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
resolution = 0.08

image_data = ImageData(full_image_path)

annotations_folder_path = os.path.join(
    Folders.CROPPED_ANNOTS.value, image_data.base_name
)

rgb_folder_path = os.path.join(Folders.CROPPED_IMAGES.value, image_data.base_name)
chm_folder_path = os.path.join(
    Folders.CHM.value,
    f"{round(resolution*100)}cm",
    "unfiltered",
    "cropped",
    image_data.coord_name,
)

means_rgb = torch.zeros(3)
means_chm = 0
count = 0

for file_name in tqdm(os.listdir(annotations_folder_path)):
    annotations_file_path = os.path.join(annotations_folder_path, file_name)
    base_name = get_file_base_name(annotations_file_path)
    rgb_path = os.path.join(rgb_folder_path, f"{base_name}.tif")
    image_rgb = io.imread(rgb_path)
    chm_path = os.path.join(chm_folder_path, f"{base_name}.tif")
    image_chm = io.imread(chm_path)

    means_rgb += np.std(image_rgb, axis=(0, 1))
    image_chm = np.where(image_chm == -9999, 0, image_chm)
    means_chm += np.std(image_chm)
    count += 1

mean_rgb = means_rgb / count
mean_chm = means_chm / count

print(f"{mean_rgb = }")
print(f"{mean_chm = }")

In [ ]:
from typing import Iterable, List


from torch.utils.data import Dataset, Sampler
from torch.utils.data.dataloader import _collate_fn_t, _worker_init_fn_t


def normalize_rgb(image_rgb: torch.Tensor) -> torch.Tensor:
    ## On the whole image
    # mean_rgb = torch.tensor([78.152, 88.417, 86.365]).view(-1, 1, 1)
    # std_rgb = torch.tensor([45.819, 42.492, 38.960]).view(-1, 1, 1)
    ## On the labeled parts
    mean_rgb = torch.tensor([77.692, 89.142, 85.816]).view(-1, 1, 1)
    std_rgb = torch.tensor([34.3328, 32.1673, 27.6653]).view(-1, 1, 1)
    return (image_rgb - mean_rgb) / std_rgb


def normalize_chm(image_chm: torch.Tensor) -> torch.Tensor:
    image_chm = torch.where(image_chm == -9999, 0, image_chm)
    ## On the whole image
    # mean_chm = 2.4113
    # std_chm = 5.5642
    ## On the labeled parts
    mean_chm = 3.0424
    std_chm = 4.5557
    return (image_chm - mean_chm) / std_chm


# def xyxy2xywh(x):
#     """
#     Convert bounding box coordinates from (x1, y1, x2, y2) format to (x, y, width, height) format where (x1, y1) is the
#     top-left corner and (x2, y2) is the bottom-right corner.

#     Args:
#         x (np.ndarray | torch.Tensor): The input bounding box coordinates in (x1, y1, x2, y2) format.

#     Returns:
#         y (np.ndarray | torch.Tensor): The bounding box coordinates in (x, y, width, height) format.
#     """
#     if isinstance(x, list):
#         x = torch.tensor(x)
#     assert x.shape[-1] == 4, f"input shape last dimension expected 4 but input shape is {x.shape}"
#     y = torch.empty_like(x) if isinstance(x, torch.Tensor) else np.empty_like(x)  # faster than clone/copy
#     y[..., 0] = (x[..., 0] + x[..., 2]) / 2  # x center
#     y[..., 1] = (x[..., 1] + x[..., 3]) / 2  # y center
#     y[..., 2] = x[..., 2] - x[..., 0]  # width
#     y[..., 3] = x[..., 3] - x[..., 1]  # height
#     return y


class TreeDataset(Dataset):
    """Tree dataset."""

    def __init__(
        self,
        annotations_folder_path: str,  # TODO: Change to take a list of files instead (to separate train and validation)
        rgb_folder_path: str,
        chm_folder_path: str,
        labels_to_index: Dict[str, int],
        labels_to_color: Dict[str, Tuple[int, int, int]],
        transform_spatial: Callable | None = None,
        transform_pixel: Callable | None = None,
    ) -> None:
        self.annotations_list: List[str] = []
        self.bboxes: Dict[str, List[List[float]]] = {}
        self.labels: Dict[str, List[int]] = {}
        for file_name in os.listdir(annotations_folder_path):
            annotations_file_path = os.path.join(annotations_folder_path, file_name)
            base_name = get_file_base_name(annotations_file_path)
            self.annotations_list.append(base_name)
            bboxes, labels = get_bounding_boxes(annotations_file_path)
            self.bboxes[base_name] = [bbox.as_list() for bbox in bboxes]
            self.labels[base_name] = [labels_to_index[label] for label in labels]

        self.rgb_folder_path = rgb_folder_path
        self.chm_folder_path = chm_folder_path
        self.labels_to_index = labels_to_index
        self.labels_to_str = {value: key for key, value in self.labels_to_index.items()}
        self.labels_to_color = labels_to_color
        self.transform_spatial = transform_spatial
        self.transform_pixel = transform_pixel

    def __len__(self) -> int:
        # return len(self.landmarks_frame)
        return len(self.annotations_list)

    def get_unnormalized(self, idx) -> Dict[str, torch.Tensor]:
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()

        # Read the images
        base_name = self.annotations_list[idx]
        rgb_path = os.path.join(self.rgb_folder_path, f"{base_name}.tif")
        image_rgb = io.imread(rgb_path)
        chm_path = os.path.join(self.chm_folder_path, f"{base_name}.tif")
        image_chm = io.imread(chm_path)

        # Get bboxes and labels
        bboxes = self.bboxes[base_name]
        labels = self.labels[base_name]

        # Apply the spatial transform to the two images, bboxes and labels
        if self.transform_spatial is not None:
            transformed_spatial = transform_spatial(
                image=image_rgb,
                image_chm=image_chm,
                bboxes=bboxes,
                class_labels=labels,
            )
            image_rgb = transformed_spatial["image"]
            image_chm = transformed_spatial["image_chm"]
            bboxes = transformed_spatial["bboxes"]
            labels = transformed_spatial["class_labels"]

        # Apply the pixel transform the to RGB image
        if transform_pixel is not None:
            transformed = transform_pixel(image=image_rgb)
            image_rgb = transformed["image"]

        to_tensor = ToTensorV2()

        sample = {
            "image_rgb": to_tensor(image=image_rgb)["image"],
            "image_chm": to_tensor(image=image_chm)["image"],
            "bboxes": torch.tensor(bboxes),
            "labels": torch.tensor(labels),
            "image_index": idx,
        }
        return sample

    def __getitem__(self, idx):
        sample = self.get_unnormalized(idx)
        sample["image_rgb"] = normalize_rgb(sample["image_rgb"])
        sample["image_chm"] = normalize_chm(sample["image_chm"])

        return sample

    def get_rgb_image(self, idx) -> np.ndarray:
        base_name = self.annotations_list[idx]
        rgb_path = os.path.join(self.rgb_folder_path, f"{base_name}.tif")
        image_rgb = io.imread(rgb_path)
        return image_rgb


def custom_collate_fn(batch):
    # Initialize lists to hold the extracted components
    rgb_images = []
    chm_images = []
    bboxes = []
    labels = []
    indices = []
    image_indices = []

    # Iterate through the batch
    for i, item in enumerate(batch):
        # Extract the components from the dictionary
        rgb_image = item["image_rgb"]
        chm_image = item["image_chm"]
        bbox = item["bboxes"]
        label = item["labels"]
        image_index = item["image_index"]

        # Append the extracted components to the lists
        rgb_images.append(rgb_image)
        chm_images.append(chm_image)
        bboxes.append(bbox)
        labels.append(label)
        indices.extend([i] * bbox.shape[0])
        image_indices.append(image_index)

    # Convert the lists to tensors and stack them
    rgb_images = torch.stack(rgb_images, dim=0)
    chm_images = torch.stack(chm_images, dim=0)
    bboxes = torch.cat(bboxes).to(torch.float32)
    labels = torch.cat(labels)
    indices = torch.tensor(indices)
    image_indices = torch.tensor(image_indices)

    batch = {
        "image_rgb": rgb_images,
        "image_chm": chm_images,
        "bboxes": bboxes,
        "labels": labels,
        "indices": indices,
        "image_indices": image_indices,
    }

    return batch


class TreeDataLoader(DataLoader):
    def __init__(
        self,
        dataset: TreeDataset,
        batch_size: int | None = 1,
        shuffle: bool | None = None,
        sampler: Sampler | Iterable | None = None,
        batch_sampler: Sampler[List] | Iterable[List] | None = None,
        num_workers: int = 0,
        collate_fn: Callable[[List], Any] | None = None,
        pin_memory: bool = False,
        drop_last: bool = False,
        timeout: float = 0,
        worker_init_fn: Callable[[int], None] | None = None,
        multiprocessing_context=None,
        generator=None,
        *,
        prefetch_factor: int | None = None,
        persistent_workers: bool = False,
        pin_memory_device: str = "",
    ):
        self.dataset: TreeDataset = dataset
        super().__init__(
            dataset,
            batch_size,
            shuffle,
            sampler,
            batch_sampler,
            num_workers,
            collate_fn,
            pin_memory,
            drop_last,
            timeout,
            worker_init_fn,
            multiprocessing_context,
            generator,
            prefetch_factor=prefetch_factor,
            persistent_workers=persistent_workers,
            pin_memory_device=pin_memory_device,
        )

In [ ]:
from collections import defaultdict


class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0.0, "count": 0, "avg": 0.0})

    def update(self, metric_name: str, val: float, count: int = 1):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += count
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                f"{metric_name}: {metric["avg"]:.{self.float_precision}f}"
                for (metric_name, metric) in self.metrics.items()
            ]
        )


def perfect_preds(
    gt_bboxes: torch.Tensor,
    gt_classes: torch.Tensor,
    gt_indices: torch.Tensor,
    batch_size: int,
):
    extracted_bboxes = [[]] * batch_size
    extracted_classes = [[]] * batch_size
    for bbox_idx, image_idx in enumerate(gt_indices):
        slice_bboxes = gt_bboxes[bbox_idx]
        extracted_bboxes[image_idx].append(slice_bboxes)
        slice_classes = gt_classes[bbox_idx].long()
        extracted_classes[image_idx].append(slice_classes)
    scores = [
        20 * nn.functional.one_hot(torch.tensor(cls), num_classes=5) - 0.5
        for cls in extracted_classes
    ]
    prefect_preds = [
        torch.cat((torch.tensor(bboxes), classes), dim=1).permute((1, 0)).unsqueeze(0)
        for bboxes, classes in zip(extracted_bboxes, scores)
    ]
    perfect_preds = torch.cat(
        [
            torch.cat(
                (
                    pred,
                    torch.zeros(
                        (pred.shape[0], pred.shape[1], 8400 - pred.shape[2])
                    ).to(pred.device),
                ),
                dim=2,
            )
            for pred in prefect_preds
        ]
    )
    # print(f"{perfect_preds.shape = }")
    return perfect_preds


def print_current_memory():
    if torch.cuda.is_available():
        current_memory_usage_bytes = torch.cuda.memory_allocated()
        current_memory_usage_megabytes = current_memory_usage_bytes / (1024 * 1024)
        print(f"Current GPU memory usage: {current_memory_usage_megabytes:.2f} MB")
    else:
        print("CUDA is not available.")


def train(
    train_loader: TreeDataLoader,
    model: AMF_GD_YOLOv8,
    optimizer: torch.optim.Optimizer,
    epoch: int,
    device: torch.device,
    accumulation_steps: int,
    running_accumulation_step: int,
) -> int:
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader)
    for i, data in enumerate(stream, start=running_accumulation_step):
        image_rgb: torch.Tensor = data["image_rgb"]
        image_chm: torch.Tensor = data["image_chm"]
        gt_bboxes: torch.Tensor = data["bboxes"]
        gt_classes: torch.Tensor = data["labels"]
        gt_indices: torch.Tensor = data["indices"]

        image_rgb = image_rgb.to(device, non_blocking=True)
        image_chm = image_chm.to(device, non_blocking=True)
        gt_bboxes = gt_bboxes.to(device, non_blocking=True)
        gt_classes = gt_classes.to(device, non_blocking=True)
        gt_indices = gt_indices.to(device, non_blocking=True)

        output = model(image_rgb, image_chm)
        print(f"@@@@@ : {gt_bboxes.mean() = }")
        total_loss = model.compute_loss(output, gt_bboxes, gt_classes, gt_indices)[0]

        batch_size = image_rgb.shape[0]
        metric_monitor.update("Loss", total_loss.item(), batch_size)

        total_loss.backward()

        # Gradient accumulation
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # optimizer.zero_grad()
        # total_loss.backward()
        # optimizer.step()

        stream.set_description(f"Epoch: {epoch}. Train.      {metric_monitor}")

    return running_accumulation_step


def validate(
    val_loader: TreeDataLoader, model: AMF_GD_YOLOv8, epoch: int, device: torch.device
):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    with torch.no_grad():
        for i, data in enumerate(stream, start=1):
            image_rgb: torch.Tensor = data["image_rgb"]
            image_chm: torch.Tensor = data["image_chm"]
            gt_bboxes: torch.Tensor = data["bboxes"]
            gt_classes: torch.Tensor = data["labels"]
            gt_indices: torch.Tensor = data["indices"]

            image_rgb = image_rgb.to(device, non_blocking=True)
            image_chm = image_chm.to(device, non_blocking=True)
            gt_bboxes = gt_bboxes.to(device, non_blocking=True)
            gt_classes = gt_classes.to(device, non_blocking=True)
            gt_indices = gt_indices.to(device, non_blocking=True)

            output = model(image_rgb, image_chm)
            print(f"@@@@@ : {gt_bboxes.mean() = }")
            total_loss = model.compute_loss(output, gt_bboxes, gt_classes, gt_indices)[
                0
            ]
            print(f"#####################{total_loss = }")

            batch_size = image_rgb.shape[0]
            metric_monitor.update("Loss", total_loss.item(), batch_size)

            stream.set_description(f"Epoch: {epoch}. Validation. {metric_monitor}")


def test_save_output_image(
    model: AMF_GD_YOLOv8,
    test_loader: TreeDataLoader,
    epoch: int,
    device: torch.device,
    number_images: int = 1,
):
    saved_images = 0
    number_images = min(len(test_loader), number_images)
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader, start=1):
            image_rgb: torch.Tensor = data["image_rgb"]
            image_chm: torch.Tensor = data["image_chm"]
            image_rgb = image_rgb.to(device, non_blocking=True)
            image_chm = image_chm.to(device, non_blocking=True)
            results = model.predict(image_rgb, image_chm)[2]

            initial_rgb = test_loader.dataset.get_rgb_image(data["image_indices"])
            colors_dict = test_loader.dataset.labels_to_color
            if results.boxes is not None:
                bboxes = results.boxes.xyxy.tolist()
                labels = [results.names[cls.item()] for cls in results.boxes.cls]
                scores = results.boxes.conf.tolist()
            else:
                bboxes = []
                labels = []
                scores = []

            bboxes_image = create_bboxes_image(
                image=initial_rgb,
                bboxes=bboxes,
                labels=labels,
                colors_dict=colors_dict,
                scores=scores,
                color_mode="bgr",
            )

            output_name = f"Validation_bboxes_{epoch}_{saved_images}_{len(bboxes)}.png"
            output_path = os.path.join(OUTPUT_DIR, output_name)
            cv2.imwrite(output_path, bboxes_image)

            saved_images += 1
            if saved_images >= number_images:
                break


def train_and_validate(
    model: AMF_GD_YOLOv8,
    train_dataset: TreeDataset,
    val_dataset: TreeDataset,
    lr: float,
    epochs: int,
    batch_size: int,
    num_workers: int,
    accumulate: int,
    device: torch.device,
) -> nn.Module:
    train_loader = TreeDataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )
    val_loader = TreeDataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )
    test_loader = TreeDataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=1,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda i: 1 / np.sqrt(i + 2), last_epoch=-1, verbose=True
    )

    accumulation_steps = round(accumulate / batch_size)
    running_accumulation_step = 0

    test_save_output_image(
        model=model, test_loader=test_loader, epoch=0, device=device, number_images=2
    )
    for epoch in range(1, epochs + 1):
        running_accumulation_step = train(
            train_loader,
            model,
            optimizer,
            epoch,
            device,
            accumulation_steps,
            running_accumulation_step,
        )
        print_current_memory()
        validate(val_loader, model, epoch, device)
        print_current_memory()
        if epoch % 1 == 0:
            test_save_output_image(
                model=model,
                test_loader=test_loader,
                epoch=epoch,
                device=device,
                number_images=2,
            )
            print_current_memory()
        scheduler.step()
    return model


def test_test_test(
    model: AMF_GD_YOLOv8,
    val_dataset: TreeDataset,
    epoch: int,
    device: torch.device,
    number_images: int = 1,
):
    saved_images = 0
    number_images = min(len(val_dataset), number_images)
    model.eval()
    while saved_images < number_images:
        sample = val_dataset[saved_images]
        image_rgb: torch.Tensor = sample["image_rgb"].unsqueeze(0)
        image_chm: torch.Tensor = sample["image_chm"].unsqueeze(0)
        image_rgb = image_rgb.to(device, non_blocking=True)
        image_chm = image_chm.to(device, non_blocking=True)
        output = model.forward(image_rgb, image_chm)
        print(f"{output[0].shape}")
        print(f"{output[1].shape}")
        print(f"{output[2].shape}")

        gt_bboxes: torch.Tensor = sample["bboxes"]
        gt_classes: torch.Tensor = sample["labels"]
        gt_indices: torch.Tensor = torch.zeros(gt_bboxes.shape[0])
        batch = {"cls": gt_classes, "bboxes": gt_bboxes, "batch_idx": gt_indices}

        criterion = v8DetectionLoss(model_test)
        criterion(output, batch)

        saved_images += 1


def tteeeeeeeest(
    val_loader: DataLoader,
    model: AMF_GD_YOLOv8,
    optimizer: torch.optim.Optimizer,
    epoch: int,
    device: torch.device,
):
    model.train()
    stream = tqdm(val_loader)
    # with torch.no_grad():
    for i, data in enumerate(stream, start=1):
        image_rgb: torch.Tensor = data["image_rgb"]
        image_chm: torch.Tensor = data["image_chm"]
        gt_bboxes: torch.Tensor = data["bboxes"]
        gt_classes: torch.Tensor = data["labels"]
        gt_indices: torch.Tensor = data["indices"]

        image_rgb = image_rgb.to(device, non_blocking=True)
        image_chm = image_chm.to(device, non_blocking=True)
        gt_bboxes = gt_bboxes.to(device, non_blocking=True)
        gt_classes = gt_classes.to(device, non_blocking=True)
        gt_indices = gt_indices.to(device, non_blocking=True)

        output = model.forward(image_rgb, image_chm)
        # print(f"{output[0].shape}")
        # print(f"{output[1].shape}")
        # print(f"{output[2].shape}")

        # # # Create the list for indices
        # # starts = gt_indices[:, 0].long().tolist()
        # # ends = gt_indices[:, 1].long().tolist()
        # # ranges = [end - start for start, end in zip(starts, ends)]
        # # indices = torch.cat(
        # #     [torch.full((count,), i) for i, count in enumerate(ranges)]
        # # ).to(gt_classes.device)

        # print(f"{gt_classes.shape = }")
        # print(f"{gt_bboxes.shape = }")
        # print(f"{gt_indices.shape = }")
        # batch = {"cls": gt_classes, "bboxes": gt_bboxes, "batch_idx": gt_indices}

        # criterion = v8DetectionLoss(model_test)
        # total_loss = criterion(output, batch)[0]

        total_loss = model.compute_loss(output, gt_bboxes, gt_classes, gt_indices)[0]

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

In [ ]:
print_current_memory()

import gc

gc.collect()
torch.cuda.empty_cache()
with torch.no_grad():
    torch.cuda.empty_cache()

print_current_memory()

In [ ]:
# Model

class_names = {
    0: "Tree",
    1: "Tree_unsure",
    2: "Tree_disappeared",
    3: "Tree_replaced",
    4: "Tree_new",
}

class_colors = {
    "Tree": (104, 201, 45),
    "Tree_unsure": (255, 215, 158),
    "Tree_disappeared": (158, 174, 255),
    "Tree_replaced": (255, 90, 82),
    "Tree_new": (251, 106, 225),
}

class_indices = {value: key for key, value in class_names.items()}

model = AMF_GD_YOLOv8(3, 1, device=device, scale="n", class_names=class_names).to(
    device
)

# Transformations for data augmentation

crop_size = 640
distort_steps = 30
distort_limit = 0.2

transform_spatial = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.GridDistortion(
            num_steps=distort_steps,
            distort_limit=(-distort_limit, distort_limit),
            border_mode=cv2.BORDER_CONSTANT,
            normalized=True,
            p=0.5,
        ),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.Perspective(interpolation=cv2.INTER_LINEAR, p=0.25),
    ],
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0.2,
        label_fields=["class_labels"],
    ),
    additional_targets={"image_chm": "image"},
)

transform_pixel = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
    ],
)

# Paths to the data

full_image_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
resolution = 0.08

image_data = ImageData(full_image_path)

annotations_folder_path = os.path.join(
    Folders.CROPPED_ANNOTS.value, image_data.base_name
)
rgb_folder_path = os.path.join(Folders.CROPPED_IMAGES.value, image_data.base_name)
chm_folder_path = os.path.join(
    Folders.CHM.value,
    f"{round(resolution*100)}cm",
    "unfiltered",
    "cropped",
    image_data.coord_name,
)

train_dataset = TreeDataset(
    annotations_folder_path=annotations_folder_path,
    rgb_folder_path=rgb_folder_path,
    chm_folder_path=chm_folder_path,
    labels_to_index=class_indices,
    labels_to_color=class_colors,
    transform_spatial=transform_spatial,
    transform_pixel=transform_pixel,
    # transform_spatial=None,
    # transform_pixel=None,
)

val_dataset = TreeDataset(
    annotations_folder_path=annotations_folder_path,
    rgb_folder_path=rgb_folder_path,
    chm_folder_path=chm_folder_path,
    labels_to_index=class_indices,
    labels_to_color=class_colors,
    transform_spatial=None,
    transform_pixel=None,
)

# Training parameters

lr = 1e-2
epochs = 100

batch_size = 2
num_workers = 2
accumulate = 12

train_and_validate(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
    num_workers=num_workers,
    accumulate=accumulate,
    device=device,
)

# test_test_test(model_test, val_dataset, 0, device, 1)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# test_save_output_image(
#     model=model,
#     val_dataset=val_dataset,
#     epoch=0,
#     device=device,
#     number_images=2,
# )
# tteeeeeeeest(
#     val_loader=DataLoader(
#         val_dataset,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=num_workers,
#         pin_memory=True,
#         collate_fn=custom_collate_fn,
#     ),
#     model=model,
#     optimizer=optimizer,
#     epoch=0,
#     device=device,
# )
# test_save_output_image(
#     model=model,
#     val_dataset=val_dataset,
#     epoch=1,
#     device=device,
#     number_images=2,
# )

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
# def print_device_of_layers(model):
#     for name, module in model.named_modules():
#         try:
#             device = next(iter(module.parameters())).device
#             # if device != torch.device("cuda:0"):
#             #     print(f"{name}: {device}")
#             print(f"{name}: {device}")
#         except StopIteration:
#             pass  # Skip modules without parameters


# print_device_of_layers(model)

for name, module in model.named_modules():
    try:
        print(f"{name}: {next(iter(module.parameters()))[0]}")
    except StopIteration:
        pass  # Skip modules without parameters

In [ ]:
rgb_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
chm_path = "../data/chm/8cm/filtered/full/122000_484000.tif"
# rgb_path = "../data/images/cropped/2023_122000_484000_RGB_hrl/0_0_640_640.tif"
# chm_path = "../data/chm/8cm/filtered/cropped/122000_484000/0_0_640_640.tif"


image_rgb = io.imread(rgb_path)
image_chm = io.imread(chm_path)

# RGB normalization

# rgb_mean = np.zeros(3, dtype=np.float32)
# rgb_std = np.zeros(3, dtype=np.float32)
# for i in range(3):
#     rgb_mean[i] = np.mean(image_rgb[:,:,i])
#     rgb_std[i] = np.std(image_rgb[:,:,i])
# print(f"{rgb_mean = }")
# print(f"{rgb_std = }")

image_rgb = normalize_rgb(image_rgb)

# CHM normalization

# chm_mean = np.mean(image_chm)
# chm_std = np.std(image_chm)
# print(f"{chm_mean = }")
# print(f"{chm_std = }")

image_chm = normalize_chm(image_chm)

### Transformation tests

In [ ]:
crop_size = 640
distort_steps = 30
distort_limit = 0.2

transform = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
    ],
)

transform_2 = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        # A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        # A.Normalize(p=1.0),
    ],
)

image_path = "../data/images/cropped/2023_122000_484000_RGB_hrl/0_0_640_640.tif"
# image_path = "../data/flat-design-abstract-outline-background_23-2150616456.jpg"

t = time()

image = io.imread(image_path)
# image = (image - rgb_mean) / rgb_std
# image = image.astype(np.float32)
print(f"{image.dtype = }")

print(f"Time elapsed: {round(time() - t, 3)}")

number_tests = 1

t = time()

for i in range(number_tests):
    r = random.random()
    random.seed(r)
    transformed_image = transform(image=image)["image"]
    random.seed(r)
    transformed_image_2 = transform_2(image=image)["image"]

    output_name = f"Augmentation_test_{i}.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    io.imsave(output_path, transformed_image)

    output_name = f"Augmentation_test_{i}_2.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    io.imsave(output_path, transformed_image_2)

print(f"Time elapsed: {round(time() - t, 3)}")